# Serverless Deep Learning

In [1]:
from keras_image_helper import create_preprocessor

import tflite_runtime.interpreter as tflite

In [3]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

image_url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(image_url)

Now, we load the model we previously created with tf-lite:

In [4]:
interpreter = tflite.Interpreter(model_path='clothing-model-v4.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


To be able to use the model, we need to get its input (where X will go) and the output
(where we get the predictions from):

In [5]:
# Gets the input: the part of the network that takes in the array X
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

# Gets the output: the part of the network with final predictions
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

To apply the model, take the X we previously prepared, put it into the input, invoke
the interpreter, and get the results from the output:

In [11]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index) # This array contains the predictions

# Labels for each class
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Final results
results = dict(zip(labels, preds[0]))

results

{'dress': -1.8798647,
 'hat': -4.7563086,
 'longsleeve': -2.359531,
 'outwear': -1.0892599,
 'pants': 9.903781,
 'shirt': -2.8261786,
 'shoes': -3.648309,
 'shorts': 3.241155,
 'skirt': -2.6120946,
 't-shirt': -4.8520336}

The model correctly predicted the 'pants' class.

### Code for the lambda function
Now, we are going to put all this code together into a single script, `lambda_function.py`.

Then, we create a Dockerfile and a Docker image. After building and running it, we can test it using the cell bellow, or the `test.py` script in this directory. 

In [18]:
import requests

data = {
    "url": "http://bit.ly/mlbookcamp-pants"
}

url = "http://localhost:8080/2015-03-31/functions/function/invocations"
results = requests.post(url, json=data).json()

results

{'dress': -1.8798644542694092,
 'hat': -4.7563090324401855,
 'longsleeve': -2.359534740447998,
 'outwear': -1.089264154434204,
 'pants': 9.903782844543457,
 'shirt': -2.8261802196502686,
 'shoes': -3.648308753967285,
 'shorts': 3.241154909133911,
 'skirt': -2.6120963096618652,
 't-shirt': -4.8520355224609375}